In [5]:
import os

In [6]:
%pwd

'c:\\Users\\hp\\Documents\\machine_leanrning\\Deep Learning\\MLOPS\\Kidney-Disease-Classification-MLOPS\\research'

In [7]:
os.chdir("../")

In [8]:
%pwd

'c:\\Users\\hp\\Documents\\machine_leanrning\\Deep Learning\\MLOPS\\Kidney-Disease-Classification-MLOPS'

In [9]:
import dagshub
dagshub.init(repo_owner='am-alibee', repo_name='Kidney-Disease-Classification-MLOPS', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Accessing as am-alibee

Initialized MLflow to track repo "am-alibee/Kidney-Disease-Classification-MLOPS"

Repository am-alibee/Kidney-Disease-Classification-MLOPS initialized!

c:\Users\hp\miniconda3\envs\chest_cancer_mlops_py310\lib\site-packages\mlflow\utils\requirements_utils.py:12: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [10]:
import tensorflow as tf


In [11]:
model = tf.keras.models.load_model("artifacts/training/model.h5")
model

In [13]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_to_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: int
    params_batch_size: int


In [14]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [ ]:
d

In [16]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [ ]:
class Evaluation:
    def __init__(self, config):
        self.config=config

    def _valid_generator(self):
        
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1], 
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_to_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()
        
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )

            # Model registry does not work with file type
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="MobileNetV2")
            else:
                mlflow.keras.log_model(self.model, "model")


In [ ]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.save_score()
    evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2026-01-08 15:00:54,400]: INFO: common: yaml file: config\config.yaml loaded successfully
[2026-01-08 15:00:54,565]: INFO: common: yaml file: params.yaml loaded successfully
[2026-01-08 15:00:54,582]: INFO: common: created directory at: artifacts
Found 12 images belonging to 4 classes.
3/3 [==============================] - 1s 125ms/step - loss: 1.5535 - accuracy: 0.5833


2026/01/08 15:01:22 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2026-01-08 15:02:03,572]: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 53). These functions will not be directly callable after loading.
INFO:tensorflow:Assets written to: C:\Users\hp\AppData\Local\Temp\tmpej8qsu0i\model\data\model\assets
[2026-01-08 15:02:16,400]: INFO: builder_impl: Assets written to: C:\Users\hp\AppData\Local\Temp\tmpej8qsu0i\model\data\model\assets


Registered model 'MobileNetV2' already exists. Creating a new version of this model...
2026/01/08 15:04:22 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: MobileNetV2, version 3
Created version '3' of model 'MobileNetV2'.
